In [2]:
import random
n=12
V = [(random.uniform(5,75),random.uniform(5,55)) for i in range(n)]
import matplotlib.pyplot as plt
alpha=2

In [3]:
alpha=2
def t_cost(s,t): return ((s[0]-t[0])**2+(s[1]-t[1])**2)**(0.5)
def d_cost(s,t): return (1/alpha)*(((s[0]-t[0])**2+(s[1]-t[1])**2)**(0.5))

In [12]:
def shiwake(V):
    sashugo={}
    for p in range(n):
        qp={i:t_cost(V[p],V[i]) for i in range(n)}
        del qp[p]
        rp={i:d_cost(V[p],V[i]) for i in range(n)}
        del rp[p]
        for i in qp:
            sa=min(qp.values())-max(rp.values())
            sashugo[p]=sa
    return sashugo

In [13]:
shiwake(V)

{0: -15.512322902215784,
 1: -17.39062675481653,
 2: -11.85363279063718,
 3: -18.495593049049585,
 4: -6.516741866479878,
 5: -7.608382202245105,
 6: -14.859050470514712,
 7: -18.09511906803781,
 8: -19.614613396433988,
 9: -16.58277116183389,
 10: -20.234101397793452,
 11: -20.889603014229927}

In [ ]:
def tpath_pathcost(start,end,tpath):
    pathcost=0
    a=tpath.index(start)
    b=tpath.index(end)
    if a<b:
        for i in range(a,b):
            pathcost=pathcost+t_cost(V[tpath[i]],V[tpath[(i+1)%len(tpath)]])
    if a>b:
        for i in range(a,len(tpath)):
            pathcost=pathcost+t_cost(V[tpath[i]],V[tpath[(i+1)%len(tpath)]])
        for i in range(b):
            pathcost=pathcost+t_cost(V[tpath[i]],V[tpath[(i+1)%len(tpath)]])
    if a==b:
        pathcost=0
            
    return pathcost

In [ ]:
import itertools
import matplotlib.pyplot as plt
def two_opt_truck(ps,truck_nodes):
    n = len(ps)
    res = list(range(n))
    update = True
    while update:
        update = False
        for (i,j) in itertools.combinations(range(n),2):
            if ((t_cost(ps[res[i]],ps[res[i+1]])+t_cost(ps[res[j]],ps[res[(j+1)%n]]))>
                (t_cost(ps[res[i]],ps[res[j]])+t_cost(ps[res[i+1]],ps[res[(j+1)%n]]))):
                res[i+1:j+1] = res[j:i:-1]
                update = True
                
    path=[truck_nodes[i] for i in res]

    return path

In [ ]:
def dp_truck(ps,truck_nodes):
    length = {} # length[(u,S)]: u を始点とし S の点すべてを回る最小経路長、ディクショナリ
    route = {} # route[(u,S)]: 最小経路長を達成するためのルート、ディクショナリ
    v = ps[0]
    n = len(ps)
    for i in range(1,n+1):
        for a in itertools.combinations(range(n),i):
            S = frozenset(a)
            for j in S:
                u = ps[j]
                if i==1:
                    length[(j,S)] = t_cost(v,u)
                    route[(j,S)] = [j]
                else:
                    Sj = S-set([j]) # S から j を除いたもの
                    k=min(Sj,key=lambda k: length[(k,Sj)]+t_cost(ps[k],u))#Sjの中でlength[(k,Sj)]+dist(ps[k],u)が最小のものを返している
                    length[(j,S)] = length[(k,Sj)]+t_cost(ps[k],u)
                    route[(j,S)] = route[(k,Sj)]+[j]
    path=[truck_nodes[i] for i in route[(0,frozenset(range(n)))]]
    
    return path 

In [14]:
#トラックのルートをdp（動的計画）で求める
#ドローンを理論的に仕分ける
import itertools
def newalgo_dp(V):
    n=len(V)
    ndmin=2
    ndmax=n//2
    D={}
    for nd in range(ndmin,ndmax+1):
#         length={i:t_cost(c,V[i]) for i in range(n)}
        length=shiwake(V)
        drone_nodes=[]
        truck_nodes=[]
        T={}
        for i in range(nd):
            drone_nodes.append(max(length, key=length.get))
            del length[max(length, key=length.get)]
        truck_nodes=list(set(range(n))-set(drone_nodes))
        
        
        
        
        
        t=[V[i] for i in truck_nodes]
        tpath=dp_truck(t,truck_nodes)
        for drone in drone_nodes:
            for (start,end) in itertools.permutations(tpath,2):
                drone_cost=d_cost(V[start],V[drone])+d_cost(V[drone],V[end])
                truck_cost=tpath_pathcost(start,end,tpath)
                T[start,drone,end]=max(drone_cost,truck_cost)



        tc=[a for a in itertools.combinations(tpath,nd)]
        dc=[b for b in itertools.permutations(drone_nodes,nd)]
        for comb in itertools.product(tc,dc):
            d=0
            for j in range(nd):
                d=d+T[comb[0][j],comb[1][j],comb[0][(j+1)%nd]]
            D[comb]=d,tpath
                
    res=min(D.values()) 
    path=min(D,key=D.get)
        
        
    return res,path